In [1]:
import cv2
import numpy as np
from IPython.display import clear_output, display
import os
import random
import string
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K


In [27]:
import datetime as dt
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard, CSVLogger, TerminateOnNaN,EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.utils import image_dataset_from_directory

In [2]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [12]:
CLASSES=["Apple Scab", "Apple Black Rot", "Apple Cedar Apple Rust", "Apple Healthy", "Corn Maize Cercospora Leaf Spot Gray Leaf Spot", "Corn Maize Common Rust", "Corn Maize Healthy", "Corn Maize Northern Leaf Blight", "Grape Black Rot", "Grape Esca Black Measles", "Grape Healthy", "Grape Leaf Blight Isariopsis Leaf Spot", "Potato Early Blight", "Potato Healthy", "Potato Late Blight", "Tomato Bacterial Spot", "Tomato Early Blight", "Tomato Healthy", "Tomato Late Blight", "Tomato Leaf Mold", "Tomato Septoria Leaf Spot", "Tomato Spider Mites Two-Spotted Spider Mite", "Tomato Target Spot", "Tomato Mosaic Virus", "Tomato Yellow Leaf Curl Virus"]
INPUT_SIZE=(256,256)
BATCH_SIZE=19

In [6]:
def pre_process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

In [26]:
training_data = image_dataset_from_directory("dataset/train", batch_size=BATCH_SIZE, labels="inferred", class_names=CLASSES, image_size=INPUT_SIZE,label_mode="categorical").map(pre_process)

Found 30647 files belonging to 25 classes.


In [ ]:
validation_data = image_dataset_from_directory("dataset/train", batch_size=BATCH_SIZE, labels="inferred", class_names=CLASSES, image_size=INPUT_SIZE,label_mode="categorical", validation_split=0.2,subset="validation").map(pre_process)

In [25]:
test_data = tf.keras.utils.image_dataset_from_directory("Dataset/test", batch_size=BATCH_SIZE, labels="inferred", class_names=CLASSES, image_size=INPUT_SIZE,label_mode="categorical").map(pre_process)

Found 250 files belonging to 25 classes.


In [ ]:
filepath = "models/model-{epoch:02d}-{val_acc:.2f}.hdf5"
callbacks = [ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch'), 
             TensorBoard(log_dir='logs', histogram_freq=0, write_graph=True, write_images=False, update_freq='epoch', profile_batch=2, embeddings_freq=0, embeddings_metadata=None), 
             CSVLogger( "training_logs/{}.csv".format(dt.datetime.now().strftime("%d/%m/%Y_%H%M%S")) , separator=',', append=False),
             TerminateOnNaN(),
             EarlyStopping(monitor='loss', patience=3)]

In [22]:
model = tf.keras.models.Sequential()
model.add(Conv2D(8, (3, 3), padding="same", activation='relu', input_shape=(256, 256, 3)))
model.add(Conv2D(16, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
model.add(Conv2D(32, (3, 3), padding="same", activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(25,activation="softmax"))
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(metrics=['accuracy'],loss="categorical_crossentropy",optimizer=opt)

In [23]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 256, 256, 8)       224       
                                                                 
 conv2d_13 (Conv2D)          (None, 256, 256, 16)      1168      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 128, 128, 16)     0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 128, 128, 32)      4640      
                                                                 
 conv2d_15 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                  

In [21]:
history = model.fit(training_data, steps_per_epoch=20,epochs=500,verbose=1, validation_split=0.2)

Epoch 1/15
10/10 [==============================] - 5s 453ms/step - loss: 3.3605 - accuracy: 0.0632 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 2/15
10/10 [==============================] - 5s 463ms/step - loss: 3.0760 - accuracy: 0.1895 - f1_m: 0.0000e+00 - precision_m: 0.0000e+00 - recall_m: 0.0000e+00
Epoch 3/15
10/10 [==============================] - 5s 463ms/step - loss: 2.6777 - accuracy: 0.3000 - f1_m: 0.1395 - precision_m: 0.2929 - recall_m: 0.0947
Epoch 4/15
10/10 [==============================] - 5s 470ms/step - loss: 2.6674 - accuracy: 0.2579 - f1_m: 0.0885 - precision_m: 0.3550 - recall_m: 0.0526
Epoch 5/15
10/10 [==============================] - 5s 484ms/step - loss: 2.7463 - accuracy: 0.2105 - f1_m: 0.1749 - precision_m: 0.5683 - recall_m: 0.1053
Epoch 6/15
10/10 [==============================] - 5s 479ms/step - loss: 2.5172 - accuracy: 0.2684 - f1_m: 0.2524 - precision_m: 0.8183 - recall_m: 0.1526
Epoch 7/15
10/10 [======================